Firstly, check whether your computer can use the GPU:

In [1]:
import torch

use_cuda = torch.cuda.is_available()

Then, load data from hard disk or public database.
Here, I will download data from MINST

In [2]:
import torchvision.datasets as dset
import torchvision.transforms as transforms

# construct transform object to normalize data
# transforms.ToTensor() used to change numpy->tensor
# transforms.Normalize((0.5,), (1, )): normalize data with mean = 0.5, std = 1. 
# Only one channel, we only need one pair of them
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1, ))])

#if not exist dataset folder './data', download from torch.datasets
train_set = dset.MNIST(root='data', train=True, transform=trans, download=True)
test_set = dset.MNIST(root='data', train=False, transform=trans, download=True)

batch_size = 100
train_iter = torch.utils.data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_iter = torch.utils.data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

print("==> total training batch number: {}".format(len(train_iter)))
print("==> total testing batch number: {}".format(len(test_iter)))

==> total training batch number: 600
==> total testing batch number: 100


Then, build the model

In [3]:
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Convlutional layer 1:
        # input size: 28 * 28 * 1
        # kernal size: 1 * 1 * 6
        # output size: 28 * 28 * 6
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=1)
            
        # pooling layer 1:
        # pooling size: 2
        # pooling type: max pooling
        # input size: 28 * 28 * 6
        # output size: 14 * 14 * 6
        self.pool1 = nn.MaxPool2d(2)
            
        # Convolutional layer 2:
        # input size: 14 * 14 * 6
        # kernal size: 5 * 5 * 16
        # output size: 10 * 10 * 16
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
            
        # Pooling layer 2:
        # pooling size: 2, max
        # input size: 10 * 10 * 16
        # output size: 5 * 5 * 16
        self.pool2 = nn.MaxPool2d(2)
            
        # fully connection layer1:
        self.fc1 = nn.Linear(5 * 5 * 16, 120)
            
        # fully connection layer2:
        self.fc2 = nn.Linear(120, 84)
            
        # fully connection layer3:
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        # flaten the 16 dimensional data to one dimension
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


        

Create an instance of the CNN module

In [4]:
import torch.optim as optim
cnn_network = CNN()
if use_cuda:
    cnn_network = cnn_network.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_network.parameters(), lr=0.01, momentum=0.9)

Training the cnn_network:

In [5]:
# Traning the CNN network:
epochs = 5
for epoch in range(epochs):
    avg_loss = 0
    for batch_index, (X, target) in enumerate(train_iter):
        # no need to transfer type to tensor
        # because we already use transform object to transform.toTensor when loading the MINST data
        
        if use_cuda:
            X, target = X.cuda(), target.cuda()
        
        # clear gradient for each batch
        optimizer.zero_grad()
        
        # feed forward:
        output = cnn_network.forward(X)
        loss = criterion(output, target)
        avg_loss += loss.item()
        
        # backpropagation
        loss.backward()  # calculate error backpropagation and delta w
        optimizer.step()  # updare all parameters
        
        # print loss:
        if (batch_index + 1) % 100 == 0 or (batch_index + 1) == len(train_iter):
            print("epoch: {}, index: {}, training loss: {:.06f}".format(
                epoch + 1, batch_index + 1, avg_loss / (batch_index + 1)))
            
            

epoch: 1, index: 100, training loss: 2.300464
epoch: 1, index: 200, training loss: 2.297294
epoch: 1, index: 300, training loss: 2.287309
epoch: 1, index: 400, training loss: 2.132568
epoch: 1, index: 500, training loss: 1.827121
epoch: 1, index: 600, training loss: 1.580111
epoch: 2, index: 100, training loss: 0.237277
epoch: 2, index: 200, training loss: 0.213630
epoch: 2, index: 300, training loss: 0.193218
epoch: 2, index: 400, training loss: 0.179362
epoch: 2, index: 500, training loss: 0.169080
epoch: 2, index: 600, training loss: 0.160550
epoch: 3, index: 100, training loss: 0.111606
epoch: 3, index: 200, training loss: 0.102187
epoch: 3, index: 300, training loss: 0.097299
epoch: 3, index: 400, training loss: 0.095118
epoch: 3, index: 500, training loss: 0.090841
epoch: 3, index: 600, training loss: 0.087964
epoch: 4, index: 100, training loss: 0.082587
epoch: 4, index: 200, training loss: 0.075017
epoch: 4, index: 300, training loss: 0.072330
epoch: 4, index: 400, training los

Test the performance on test set:

In [7]:
# Testing
correct_cnt = 0
total_cnt = 0
for batch_index, (X, target) in enumerate(test_iter):
    if use_cuda:
        X, target = X.cuda(), target.cuda()
    
    # get the ouput:
    output = cnn_network.forward(X)

    # select the class with maximum probability:
    _, pred_lable = torch.max(output, 1)
    
    total_cnt += X.size()[0]
    correct_cnt += (target == pred_lable).sum()
    
    # print the accuracy:
    if (batch_index + 1) % 100 == 0 or (batch_index + 1) == len(test_iter):
        print("epoch: {}, index: {}, test accuracy: {:.03f}".format(
            epoch + 1, batch_index + 1, correct_cnt * 1.0 / total_cnt))
    

epoch: 5, index: 100, test accuracy: 0.978
